In [9]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_SUBSCRIPTION_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"]
)

model_name = os.environ["AZURE_OPENAI_MODEL"]
deployment = os.environ["AZURE_OPENAI_DEPLOYMENT"]

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "I am going to Paris, what should I see?",
        }
    ],
    max_completion_tokens=100000,
    model=deployment
)

print(response.choices[0].message.content)

Paris is a treasure trove of art, history, and culture, so you’ll definitely have plenty to explore. Here are some must-see highlights:

1. Eiffel Tower – No visit to Paris is complete without marveling at this iconic landmark. Whether you admire it from below, have a picnic in the Champ de Mars, or take a trip up one of its levels, the views of Paris are unforgettable.

2. Louvre Museum – Home to thousands of works of art, including Leonardo da Vinci’s Mona Lisa, the Louvre is a must for art lovers. It’s vast, so consider picking a few sections that interest you the most or joining a guided tour.

3. Musée d'Orsay – Housed in a former railway station, this museum is famous for its extensive collection of Impressionist and post-Impressionist masterpieces by artists like Monet, Van Gogh, and Renoir.

4. Notre-Dame de Paris Area – While the famous cathedral has been under restoration since the fire in 2019, the area around it is steeped in history and worth a stroll. You can admire its f

In [11]:
from langgraph.graph import StateGraph, END
from pydantic import BaseModel

# 1. Define the state schema
class AgentState(BaseModel):
    input: str
    response: str = ""

# 2. Create the state graph with the schema
builder = StateGraph(AgentState)

# 3. Define your LangGraph node function
def respond(state: AgentState) -> AgentState:
    output = llm.invoke(state.input)
    return AgentState(input=state.input, response=output.content)

# 4. Add node and configure the graph
builder.add_node("generate", respond)
builder.set_entry_point("generate")
builder.set_finish_point("generate")

# 5. Compile and run
graph = builder.compile()
result = graph.invoke({"input": "Hello, world!"})
print(result)

{'input': 'Hello, world!', 'response': 'Hello, world! How can I help you today?'}


In [13]:
from langchain_openai import AzureChatOpenAI
from langgraph.graph import StateGraph, END


llm = AzureChatOpenAI(
    azure_deployment= os.environ["AZURE_OPENAI_DEPLOYMENT"],  # or your deployment
    api_version= os.environ["AZURE_OPENAI_API_VERSION"],  # or your api version
    # other params...
)

In [16]:
from langgraph.prebuilt import create_react_agent

def get_weather(city: str) -> str:  
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"


agent = create_react_agent(
    model=llm,  
    tools=[get_weather],  
    prompt="You are a helpful assistant"  
)

# Run the agent
agent.invoke({"messages": "what is the weather in sf"})

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='18f6fbef-a975-43a7-9e9f-1987a52bafb9'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LSkE6Lei37VvJUgF0EPkoSa9', 'function': {'arguments': '{"city": "San Francisco"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 152, 'prompt_tokens': 59, 'total_tokens': 211, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o3-mini-2025-01-31', 'system_fingerprint': 'fp_ded0d14823', 'id': 'chatcmpl-BO5IzxpYMdRrxtDlIEvrUgQmoifjI', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtere

# Static Prompt

In [19]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=llm,
    tools=[get_weather],
    # A static prompt that never changes
    prompt="Never answer questions about the weather."
)

agent.invoke(
    {"messages": "what is the weather in sf"},
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='dd8d3c7e-7286-4f22-86d7-500e0ebb9ae2'),
  AIMessage(content="I'm sorry, but I can't help with that.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 278, 'prompt_tokens': 60, 'total_tokens': 338, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o3-mini-2025-01-31', 'system_fingerprint': 'fp_ded0d14823', 'id': 'chatcmpl-BO5NjsmBeGnJj310I7sIpKYfuuNYD', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'se

In [ ]:
# Dynamic Prompt
from langchain_core.messages import AnyMessage
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.prebuilt import create_react_agent

def prompt(state: AgentState, config: RunnableConfig) -> list[AnyMessage]:
    